In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [2]:
%load_ext nb_mypy

Version 1.0.5


# Brute Force CSP Solver

In [3]:
from typing import TypeVar

In [4]:
Value      = TypeVar('Value')
Element    = TypeVar('Element')
Variable   = str
Formula    = str
CSP        = tuple[set[Variable], set[Value], set[Formula]]
Assignment = dict[Variable, Value]

## Utility Functions

The function `arb(S)` takes a set `S` as input and returns an arbitrary element from 
this set.  The set `S` is not changed.

In [5]:
def arb(S: set[Element]) -> Element:
    for x in S:
        return x
    return None # type: ignore

The global variable `gCounter` is used to count the number of variable assignments that have been tried until a solution is found.

In [6]:
gCounter: int = 0

We need the following forward declarations for the type checker.

In [7]:
def brute_force_search(Assignment, CSP) -> Assignment | None:
    return None 

In [8]:
def check_all_constraints(A: Assignment, Cnstrnts: set[Formula]) -> bool:
    return None # type: ignore

The procedure `solve(P)` takes a *constraint satisfaction problem* 
`P` as input.  Here `P` is a triple of the form 
$$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where 
- $\mathtt{Vars}$ is a set of strings which serve as *variables*,
- $\mathtt{Values}$ is a set of *values* that can be assigned 
  to the variables in $\mathtt{Vars}$.
- $\mathtt{Constraints}$ is a set of formulas that are represented as *Boolean expressions*.  
  Each of these formulas is  called a *constraint* of $\mathcal{P}$.
  
The sole purpose of the function `solve` is to call the function `brute_force_search`, which needs an additional argument.  This argument is a *partial variable assignment* that is initially empty.  Every recursive iteration of the function `brute_force_search` assigns one additional variable.

In [16]:
def solve(P: CSP) -> Assignment | None:
    return brute_force_search({}, P)

The function `brute_force_search` takes two arguments:
- `Assignment` is a <em style="color:blue">partial variable assignment</em> that is
   represented as a dictionary.  Initially, this assignment will be the  empty
   dictionary.  Every recursive call of `brute_force_search` adds the assignment of one 
   variable to  the given assignment. 
- `csp` is a constraint satisfaction problem.

The implementation of `brute_force_search` works as follows:
- If all variables have been assigned a value, the dictionary `Assignment` will have the same number of entries as the set `Variables` has elements.  Hence, in that case `Assignment` is a complete assignment of all variables and we have to test whether    all constraints are satisfied.  This is done using the auxiliary procedure `check_all_constraints`.
- Otherwise, we pick a variable that has not been assigned and recursively try to assign all possible 
  values for this variable.

In [9]:
def brute_force_search(A: Assignment, csp: CSP) -> Assignment | None:
    Variables, Values, Constraints = csp
    if len(A) == len(Variables): # all variables have been assigned
        global gCounter
        gCounter += 1
        if check_all_constraints(A, Constraints):
            return A             # A is a solution
        else:
            return None          # A does not solve the problem
    var = arb(Variables - A.keys())
    for value in Values:
        NewAss      = A.copy()
        NewAss[var] = value
        if Result := brute_force_search(NewAss, csp):
            return Result
    return None

The function `check_all_constraints` takes two arguments:
- `Assignment` is a variable assignment that is represented as a dictionary.
- `Constraints` is a set of Boolean Python expressions.
The function returns `True` iff all these expressions evaluate as `True` using
the given `Assignment`.

Below, we have to create a copy of `Assignment` since the function `eval` modifies the assignment given to it.

In [10]:
def check_all_constraints(A: Assignment, Cnstrnts: set[Formula]) -> bool:
    CA = A.copy()
    return all(eval(f, CA) for f in Cnstrnts)

## Map Coloring

In [11]:
%run MapColoring.ipynb

The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy
2187
  Cloning https://github.com/reclinarka/problem_visuals to /private/var/folders/q9/qftgdjx91wx4s5jcqkfz5bd00000gn/T/pip-req-build-fjute__9
  Running command git clone --filter=blob:none --quiet https://github.com/reclinarka/problem_visuals /private/var/folders/q9/qftgdjx91wx4s5jcqkfz5bd00000gn/T/pip-req-build-fjute__9
  Resolved https://github.com/reclinarka/problem_visuals to commit 5a7abd2897400e33220fd32be23a2e4f70661f21
  Preparing metadata (setup.py) ... done


In [12]:
%unload_ext nb_mypy

In [13]:
P = map_coloring_csp()
P

(['WA', 'NSW', 'V', 'NT', 'SA', 'Q', 'T'],
 {'blue', 'green', 'red'},
 {'NSW != V',
  'NT != Q',
  'NT != SA',
  'Q != NSW',
  'SA != NSW',
  'SA != Q',
  'SA != V',
  'V != T',
  'WA != NT',
  'WA != SA'})

Below is the unclored map.

In [14]:
show_solution({})

Let us compute a coloring next.

In [17]:
%%time 
gCounter=0
Solution=solve(P)
print(f"Number of different variable assignments that were tried: {gCounter}")

Number of different variable assignments that were tried: 98
CPU times: user 3.82 ms, sys: 310 µs, total: 4.13 ms
Wall time: 4.08 ms


In [18]:
show_solution(Solution) 

## N Queens Problem 

The notebook `N-Queens-Problem-CSP.ipynb` provides the function
`create_csp(n)` that returns a CSP encoding the 
*n queens puzzle*.

In [ ]:
%run NQueensProblemCSP.ipynb

In [ ]:
%unload_ext nb_mypy

In [ ]:
P = create_csp(8)
P

Brute force search takes about 9 seconds on my desktop to solve the eight queens puzzle.

In [ ]:
%%time
gCounter = 0
Solution = solve(P)
print(f'Solution = {Solution}')
print(f"Number of different variable assignments that were tried: {gCounter}")

The number of assignments of values from the set $\{1,\cdots,8\}$ to the variables `V1`, $\cdots$, `V8` is $8^8 = 16,777,216$.

In [ ]:
show_solution(Solution, 8)

In the $8$ queens problem we have to test $8^8$ different assignments.  The $7$ queens problem only has $7^7$ assignments.

In [ ]:
8**8 / 7**7

Therefore, the time needed to solve the $7$ queens problem should be smaller by a factor of $\displaystyle\frac{8^8}{7^7}$:

In [ ]:
P = create_csp(7)

In [ ]:
%%time
gCounter = 0
Solution = solve(P)
print(f'Solution = {Solution}')
print(f"Number of different variable assignments that were tried: {gCounter}")

The ratio is not exact as *brute force search* does not check all $n^n$ different valuations but rather stops as soon as a solution is found.

In [ ]:
show_solution(Solution, 7, width="40%")